Airline Delay Data- Data Cleaning and Preprocessing in Python



Step 1: Loading Dataset from Kaggle


In [2]:
!kaggle datasets download -d sriharshaeedala/airline-delay

Dataset URL: https://www.kaggle.com/datasets/sriharshaeedala/airline-delay
License(s): U.S. Government Works
airline-delay.zip: Skipping, found more recently modified local copy (use --force to force download)


Step 2: Unzipping the file

In [3]:
#Unzipping the file
import zipfile
with zipfile.ZipFile("airline-delay.zip", 'r') as zip_ref:
    zip_ref.extractall("airlines_dataset")

Step 3: Importing Pandas Library and Quick Analysis

In [4]:
import pandas as pd

df = pd.read_csv("airlines_dataset/Airline_Delay_Cause.csv")  
df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2023,8,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",89.0,13.0,2.25,1.60,...,0.0,5.99,2.0,1.0,1375.0,71.0,761.0,118.0,0.0,425.0
1,2023,8,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional",62.0,10.0,1.97,0.04,...,0.0,7.42,0.0,1.0,799.0,218.0,1.0,62.0,0.0,518.0
2,2023,8,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",62.0,10.0,2.73,1.18,...,0.0,4.28,1.0,0.0,766.0,56.0,188.0,78.0,0.0,444.0
3,2023,8,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",66.0,12.0,3.69,2.27,...,0.0,1.57,1.0,1.0,1397.0,471.0,320.0,388.0,0.0,218.0
4,2023,8,9E,Endeavor Air Inc.,ALB,"Albany, NY: Albany International",92.0,22.0,7.76,0.00,...,0.0,11.28,2.0,0.0,1530.0,628.0,0.0,134.0,0.0,768.0


In [5]:
#Check column names, data types and null counts
print(df.info())  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171666 entries, 0 to 171665
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   year                 171666 non-null  int64  
 1   month                171666 non-null  int64  
 2   carrier              171666 non-null  object 
 3   carrier_name         171666 non-null  object 
 4   airport              171666 non-null  object 
 5   airport_name         171666 non-null  object 
 6   arr_flights          171426 non-null  float64
 7   arr_del15            171223 non-null  float64
 8   carrier_ct           171426 non-null  float64
 9   weather_ct           171426 non-null  float64
 10  nas_ct               171426 non-null  float64
 11  security_ct          171426 non-null  float64
 12  late_aircraft_ct     171426 non-null  float64
 13  arr_cancelled        171426 non-null  float64
 14  arr_diverted         171426 non-null  float64
 15  arr_delay        

In [6]:
#Shows missing values per column 
print(df.isnull().sum()) 

year                     0
month                    0
carrier                  0
carrier_name             0
airport                  0
airport_name             0
arr_flights            240
arr_del15              443
carrier_ct             240
weather_ct             240
nas_ct                 240
security_ct            240
late_aircraft_ct       240
arr_cancelled          240
arr_diverted           240
arr_delay              240
carrier_delay          240
weather_delay          240
nas_delay              240
security_delay         240
late_aircraft_delay    240
dtype: int64


In [7]:
#Dropping rows with no data
df = df[df['arr_flights'].notnull()]
#Still seeing arr_del15 has N/As *delayed for more than 15 minutes
df.isnull().sum()

#For this analysis,we will assume that the delay data was unreported and put 0's in place.
df['arr_del15'].fillna(0, inplace=True)


In [8]:
#Double Checking Duplicate Data
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 0


In [9]:
#Feature Engineering/Data Cleaning




#Month/Year Column
df['Month_Year']= df['month'].astype(str) + "/" + df['year'].astype(str)

#Creating Season Column
def map_season(month):
    if month in [12,1,2]:
        return 'Winter'
    elif month in [3,4,5]:
        return 'Spring'
    elif month in [6,7,8]:
        return 'Summer'
    else:
        return 'Fall'
    
df['season']=df['month'].apply(map_season)
    


#Route ID column
df['route_id']= df['airport'] + "_" + df['carrier']

#Total delay count column
delay_count= ['carrier_ct','weather_ct','nas_ct','security_ct','late_aircraft_ct']
df['Total_delay_count']=df[delay_count].sum(axis=1)

#Delay rate column
df['delay_rate_%']= ((df['arr_del15']/df['arr_flights'])*100).astype(float).round(2)

#Average delay per filght
delay_columns_mins= ['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']
df['total_delay_minutes']=df[delay_columns_mins].sum(axis=1)
df['avg_delay_per_flight']=df['total_delay_minutes']/ df['arr_flights']


#Cancel and Divert Rates
df['cancel_rate_%'] = ((df['arr_cancelled'] / df['arr_flights']) * 100).astype(float).round(2)

df['divert_rate_%'] = ((df['arr_diverted'] / df['arr_flights']) * 100).astype(float).round(2)


#Month over month change delay_rate (grouped by route_id)
df = df.sort_values(by=['route_id', 'year', 'month'])
df['month_over_month_change'] = df.groupby('route_id')['delay_rate_%'].pct_change().round(3) * 100

#Removing month and year columns to avoid redundancy
df = df.drop(columns=['month', 'year'])

df.head()

,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,...,Month_Year,season,route_id,Total_delay_count,delay_rate_%,total_delay_minutes,avg_delay_per_flight,cancel_rate_%,divert_rate_%,month_over_month_change
170395,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",60.0,12.0,4.47,0.00,3.10,0.0,...,8/2013,Summer,ABE_9E,11.99,20.00,500.0,8.333333,0.00,0.00,NaN
170235,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",55.0,8.0,2.53,0.00,3.32,0.0,...,9/2013,Fall,ABE_9E,8.00,14.55,348.0,6.327273,1.82,0.00,-27.2
167733,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",31.0,1.0,1.00,0.00,0.00,0.0,...,10/2013,Fall,ABE_9E,1.00,3.23,29.0,0.935484,0.00,0.00,-77.8
166389,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",70.0,10.0,4.99,0.08,1.29,0.0,...,11/2013,Fall,ABE_9E,10.01,14.29,582.0,8.314286,0.00,0.00,342.4
165015,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",79.0,22.0,9.32,1.00,5.93,0.0,...,12/2013,Winter,ABE_9E,22.00,27.85,1411.0,17.860759,0.00,2.53,94.9


In [10]:
# Save to CSV file
df.to_csv("C:/Users/britt/Documents/airlines_info_cleaned.csv", index=False)

In [11]:
#Creating a db file to use in SQLliteStudio

import pandas as pd
import sqlite3
import os


db_path = os.path.abspath("C:/Users/britt/Documents/airline_delays.db")

# Connect and write the DataFrame
conn = sqlite3.connect(db_path)
df.to_sql("delays_cleaned", conn, if_exists="replace", index=False)
conn.close()

print("Database created at:", db_path)

Database created at: C:\Users\britt\Documents\airline_delays.db
